In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
from sympy import *
import pickle
import datetime
from tqdm import tqdm
import scipy.constants
import time

from sklearn.gaussian_process.kernels import WhiteKernel, Matern, RBF

from flowline_ode.plots_setup import *
from flowline_ode.sia_model import *
from flowline_ode.finite_differences import *
from flowline_ode.ode_solve import *
from flowline_ode.noise import *

In [ ]:
t_start_notebook = time.time()

In [ ]:
starting_example = "outputs/20241023_125643_n2.0.pickle"
#starting_example = "outputs/20240627_172129_n3.0.pickle"

with open(starting_example, 'rb') as f:
    results = pickle.load(f)

In [ ]:
rng = np.random.default_rng(42)

def rerun_with_noise(results, snr_db, cross_track_slope_angle_deg,
                     center_frequency=60e6, velocity=30, prf=10,
                     add_simulated_noise=True,
                     apply_stacking=True, apply_gp_smoothing=True):
    layers_t0 = results['layers_t0']
    layers_t1 = results['layers_t1']
    domain_x = results['domain_x']

    pulse_spacing = velocity / prf # m
    pulses_x = np.arange(0, domain_x, pulse_spacing)

    if add_simulated_noise:
        layers_t0_measured = add_noise_to_layers(layers_t0, snr_db, domain_x,
                                                velocity=velocity, prf=prf,
                                                center_frequency=center_frequency,
                                                cross_track_slope_angle_deg=cross_track_slope_angle_deg,
                                                rng=rng)
        layers_t1_measured = add_noise_to_layers(layers_t1, snr_db, domain_x,
                                                velocity=velocity, prf=prf,
                                                center_frequency=center_frequency,
                                                cross_track_slope_angle_deg=cross_track_slope_angle_deg,
                                                rng=rng)

    else:
        print("WARNING: Noise disabled in this simulation")
        layers_t0_measured = layers_t0
        layers_t1_measured = layers_t1

    #
    # Noise filtering
    #

    kernel = RBF(length_scale=1e3/domain_x, length_scale_bounds=(1e3, 50e3)) + WhiteKernel(1e-5, noise_level_bounds=(1e-10, 1))

    if apply_stacking:
        aperture_length = 100

        layers_t0_smoothed = simulate_stacking(layers_t0_measured, domain_x, kernel_length_m=aperture_length, velocity=velocity, prf=prf)
        layers_t1_smoothed = simulate_stacking(layers_t1_measured, domain_x, kernel_length_m=aperture_length, velocity=velocity, prf=prf)

        if apply_gp_smoothing:
            layers_t0_smoothed = gp_smoothing(layers_t0_smoothed, domain_x, x_spacing=aperture_length, initialize_with_prior_kernel=False,
                                            kernel=kernel)
            layers_t1_smoothed = gp_smoothing(layers_t1_smoothed, domain_x, x_spacing=aperture_length, initialize_with_prior_kernel=False,
                                            kernel=kernel)

    else:
        layers_t0_smoothed = layers_t0_measured
        layers_t1_smoothed = layers_t1_measured

    #
    # Layer finite difference functions
    #

    layer_dl_dx, layer_dl_dt, layer_d2l_dxdz, layer_d2l_dtdz = create_layer_finite_difference_fns(layers_t0_smoothed, layers_t1_smoothed)

    #
    # Method of characteristics solution
    #

    layer_solutions = solve_all_layers(layers_t0_smoothed, layer_d2l_dxdz,
                                    layer_d2l_dtdz, results['x'], results['z'],
                                    results['domain_x'], pulses_x, solve_args={'method': 'Radau'})

    #
    # Store results with noise
    #

    results_with_noise = {
            'n': results['n'],
            'xs': results['xs'],
            'zs': results['zs'],
            'domain_x': results['domain_x'],
            'domain_z': results['domain_z'],
            'surface': results['surface'],
            'x': results['x'],
            'z': results['z'],
            'u': results['u'],
            'w': results['w'],
            'ds_dx': results['ds_dx'],
            # layers
            'layers_t0': results['layers_t0'],
            'layers_t1': results['layers_t1'],
            'layers_t0_measured': layers_t0_measured,
            'layers_t1_measured': layers_t1_measured,
            'layers_t0_smoothed': layers_t0_smoothed,
            'layers_t1_smoothed': layers_t1_smoothed,
            'layer_solutions': layer_solutions,
            # noise
            'snr_db': snr_db,
            'cross_track_slope_angle_deg': cross_track_slope_angle_deg,
            'velocity': velocity,
            'prf': prf,
            'center_frequency': center_frequency,
        }
    
    return results_with_noise

In [ ]:
# import copy
# results_test = copy.deepcopy(results_with_noise)
# results_test.pop('layers_t0_measured')
# results_test.pop('layers_t1_measured')
# with open("outputs/test.pickle", "wb") as f:
#     pickle.dump(results_test, f)

In [ ]:
snr_values = [20, 15, 10, 5, 1]
cross_track_slope_angle_deg_values = [0, 5, 10, 15]
n_trials = 5

for snr_db in snr_values:
    for cross_track_slope_angle_deg in cross_track_slope_angle_deg_values:
        for trial_idx in range(n_trials):
            results_with_noise = rerun_with_noise(results, snr_db, cross_track_slope_angle_deg, apply_gp_smoothing=False)
            output_pickle_path = f"{starting_example}.rerun_radau_snr{snr_db}_slope{cross_track_slope_angle_deg}_trial{trial_idx}.pickle"
            print(f"Saving results to {output_pickle_path}")
            with open(output_pickle_path, 'wb') as f:
                pickle.dump(results_with_noise, f)

In [ ]:
print(f"Total elapsed time: {(time.time() - t_start_notebook)/60:.2f} minutes")